In [ ]:
def MobileNetv2(input_shape, k, alpha=1.0):
# model = MobileNetv2((224, 224, 3), 100, 1.0)
    
    inputs = Input(shape = input_shape)
    # inputs = Input(shape = (224, 224, 3))
    
    first_filter = _make_divisible(32 * alpha, 8)
    # first_filter = _make_divisible(32 * 1.0, 8)
    
    x = _conv_block(inputs, first_filters, (3,3), strides=(2,2))
    # 1. x = _conv_block((224, 224, 3), new_v, kernel=(3,3), strides=(2,2))
    # filter shape = 3x3x3x32 / input size = 224x224x3
    
    x = _inverted_residual_block(x, 16, (3,3), t=1, alpha=alpha, stride=1, n=1)
    # 2. _inverted_residual_block(x, filter=16, kernel=(3,3), t=1, alpha=1.0, stride=1, n=1)
    # 2-1. conv : filter shape = 3x3x32x16 / input size = 112x112x32
    # 2-2. depth : filter shape = 3x3x16 / input size = 112x112x16
    # 2-3. Conv2D(16, (1,1), stride 1) filter shape = 1x1x16x16 / input size = 112x112x16
    
    x = _inverted_residual_block(x, 24, (3, 3), t=6, alpha=alpha, strides=2, n=2)
    # 3. _inverted_residual_block(x, 24, (3, 3), t=6, alpha=1.0, strides=2, n=2)
    # 3-1. conv : filter shape = 3x3x16x24 / input size = 112x112x16
    # 3-2. depth : filter shape = 3x3x24 / input size = 112x112x24
    # 3-3. Conv2D : filter shape = 1x1x24x24 / input size = 56x56x24
    
    x = _inverted_residual_block(x, 32, (3, 3), t=6, alpha=alpha, strides=2, n=3)
    # 4. 
    # 4-1. conv : filter shape = 3x3x24x32 / input size = 56x56x24
    # 4-2. depth : filter shape = 3x3x32 / input size = 56x56x32
    # 4-3. Conv2D : filter shape = 1x1x32x32 / input size = 28x28x32
    
    x = _inverted_residual_block(x, 64, (3, 3), t=6, alpha=alpha, strides=2, n=4)
    # 5. 
    # 5-1. conv : filter shape = 3x3x32x64 / input size = 28x28x32
    # 5-2. depth : filter shape = 3x3x64 / input size = 28x28x64
    # 5-3. Conv2D : filter shape = 1x1x64x64 / input size = 14x14x64
    
    x = _inverted_residual_block(x, 96, (3, 3), t=6, alpha=alpha, strides=1, n=3)
    # 6.
    # 6-1. conv : filter shape = 3x3x64x96 / input size = 14x14x64
    # 6-2. depth : filter shape = 3x3x96 / input size = 14x14x96
    # 6-3. Conv2D : filter shape = 1x1x96x96 /  input size = 14x14x96
    
    x = _inverted_residual_block(x, 160, (3, 3), t=6, alpha=alpha, strides=2, n=3)
    # 7.
    # 7-1. conv : filter shape = 3x3x96x160 / input size =  14x14x96
    # 7-2. depth : filter shape = 3x3x160 /  input size = 14x14x160
    # 7-3. Conv2D : filter shape = 1x1x160x160 / input size = 7x7x160

    x = _inverted_residual_block(x, 320, (3, 3), t=6, alpha=alpha, strides=1, n=1)
    # 8. 
    # 8-1. conv : filter shape =  3x3x160x320 / input size = 7x7x160
    # 8-2. depth : filter shape = 3x3x320 / input size = 7x7x320
    # 8-3. Conv2D : filter shape = 1x1x320x320 / input size = 7x7x320 
    
    if alpha > 1.0:
        last_filters = _make_divisible(1280 * alpha, 8)
    else:
        last_filters = 1280
    
    x = _conv_block(x, last_filters, (1, 1), strides=(1, 1))
    # 9. filter shape = 1x1x320x1280 / input size = 7x7x320
    
    x = GlobalAveragePooling2D()(x)
    # 10. input size = 7x7x1280
    
    x = Reshape((1, 1, last_filters))(x)    
    x = Dropout(0.3, name='Dropout')(x)
    
    x = Conv2D(k=100, (1, 1), padding='same')(x)
    # 10. filter shape = 1x1x1280x100 / input size = 1x1x1280
    
    x = Activation('softmax', name='softmax')(x)
    output = Reshape((k,))(x)

    model = Model(inputs, output)
    # plot_model(model, to_file='images/MobileNetv2.png', show_shapes=True)

    return model
    
    